In [364]:
import torch
from torch import Tensor
from nba_api.stats.static import teams
from pandas import DataFrame
from typing import List, Tuple

In [2]:
all_teams = teams.get_teams()

In [3]:
team = [team for team in all_teams if team['abbreviation'] == 'ATL'][0]
team_id = team['id']
print(team_id)

1610612737


In [4]:
from nba_api.stats.endpoints import leaguegamefinder
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
games = gamefinder.get_data_frames()
print(games[0].head())

  SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
0     22025  1610612737               ATL  Atlanta Hawks  0022500775   
1     22025  1610612737               ATL  Atlanta Hawks  0022500765   
2     22025  1610612737               ATL  Atlanta Hawks  0022500751   
3     22025  1610612737               ATL  Atlanta Hawks  0022500735   
4     22025  1610612737               ATL  Atlanta Hawks  0022500720   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2026-02-11    ATL @ CHA  L  239  107  ...   0.767     8    33   41   28   
1  2026-02-09    ATL @ MIN  L  240  116  ...   0.786    17    30   47   20   
2  2026-02-07  ATL vs. CHA  L  239  119  ...   0.882     5    25   30   30   
3  2026-02-05  ATL vs. UTA  W  241  121  ...   0.739    19    31   50   34   
4  2026-02-03    ATL @ MIA  W  240  127  ...   0.615     8    39   47   34   

    STL  BLK  TOV  PF  PLUS_MINUS  
0   8.0    6   13  21        -3.0  
1  12.0    1   18  19     

In [5]:
# takes in the id of team A and the abbreviation of team B and returns their 5 latest matchups
def last_five_games_against_b(a_id: str, b_abr: str):
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=a_id)
    a_games = gamefinder.get_data_frames()[0]
    return a_games[a_games.MATCHUP.str.contains(b_abr)].head()


In [6]:
matchups = last_five_games_against_b(team_id, 'CHA')
matchups

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22025,1610612737,ATL,Atlanta Hawks,0022500775,2026-02-11,ATL @ CHA,L,239,107,...,0.767,8,33,41,28,8.0,6,13,21,-3.0
2,22025,1610612737,ATL,Atlanta Hawks,0022500751,2026-02-07,ATL vs. CHA,L,239,119,...,0.882,5,25,30,30,9.0,6,10,20,-7.0
28,22025,1610612737,ATL,Atlanta Hawks,0022500371,2025-12-18,ATL @ CHA,L,238,126,...,0.864,9,26,35,31,6.0,2,15,20,-7.0
38,22025,1610612737,ATL,Atlanta Hawks,0022500276,2025-11-23,ATL vs. CHA,W,242,113,...,0.750,10,27,37,32,6.0,5,7,21,3.0
75,22024,1610612737,ATL,Atlanta Hawks,0022400993,2025-03-18,ATL @ CHA,W,241,134,...,0.800,11,35,46,27,12.0,8,10,17,32.0


In [7]:
def season_from_id(season_id: str):
    start_year = season_id[-2:]
    end_year = str(int(start_year)+1)
    return f'20{start_year}-{end_year}'

In [381]:
# wrapper for empty tensor
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA not available. Using CPU.")
    device = torch.device("cpu")

def T(shape: tuple) -> torch.Tensor:
    return torch.empty(shape, device=device, dtype=torch.float32)

CUDA is available. Using GPU.


In [388]:
# wrapper for random tensor
def Tr(shape: tuple) -> torch.Tensor:
    return torch.rand(shape, device=device, dtype=torch.float32)

In [382]:
# quantities included in input tensor
num_quantities = 20

In [411]:
from nba_api.stats.endpoints import boxscoretraditionalv3
from nba_api.stats.endpoints import commonteamroster
import numpy as np

# get box scores of each player for each game in df of games of a single team
def get_box_scores(games: DataFrame, num_quantities: int, 
                   device: torch.device | None = None) -> Tuple[List[List[DataFrame]], str, str, str]:
    if not device:
        if torch.cuda.is_available():
            print("CUDA is available. Using GPU.")
            device = torch.device("cuda")
        else:
            print("CUDA not available. Using CPU.")
            device = torch.device("cpu")

    
    team_id = games['TEAM_ID'].iloc[0]
    season = season_from_id(games['SEASON_ID'].iloc[0]) # most recent

    a_roster = (
        commonteamroster.CommonTeamRoster(team_id=team_id, season=season).
        get_data_frames()[0].
        PLAYER_ID
    )

    a_roster_map = {player_id: i for i, player_id in enumerate(a_roster)}

    input = T(((len(games)), len(a_roster), num_quantities))

    for i, game in enumerate(games.itertuples()):

        game_id = game.GAME_ID
        opp_abv = game.MATCHUP[-3:]
        opp_id = teams.find_team_by_abbreviation(opp_abv)['id']

        # filter box scores by players in current roster and quantitative metrics
        box_scores = boxscoretraditionalv3.BoxScoreTraditionalV3(game_id=game_id).get_data_frames()[0]
        box_scores = box_scores[box_scores["personId"].isin(a_roster_map)]

        # reformat minutes str -> float
        time = box_scores['minutes'].str.split(":")
        mins = time.str[0].replace("", "0").astype(int)
        secs = time.str[1].fillna(0).replace("", "0").astype(int)
        box_scores['minutes'] = mins + secs / 60
        
        box_scores_quant = box_scores.iloc[:,14:]
        
        # sort box scores into input tensor using personId -> index mapping
        rows = box_scores['personId'].map(a_roster_map).to_numpy()
        input[i][rows] = torch.from_numpy(box_scores_quant.to_numpy()).to(device).float()

    return input, len(a_roster_map)
        

In [412]:
def last_t_games(id, t):
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=id)
    games = gamefinder.get_data_frames()[0]
    return games.head(t)

In [419]:
t = 10
games = last_t_games(team_id, t)
box_scores, num_players = get_box_scores(games, num_quantities, device)

In [421]:
DataFrame(Tensor.cpu(box_scores[0]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34.450001,7.0,15.0,0.467,1.0,4.0,0.250,4.0,5.0,0.80,1.0,12.0,13.0,9.0,4.0,1.0,3.0,1.0,19.0,-5.0
3,0.000000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31.966667,6.0,17.0,0.353,1.0,8.0,0.125,4.0,5.0,0.80,2.0,4.0,6.0,8.0,0.0,1.0,3.0,2.0,17.0,7.0
5,0.000000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,25.483334,9.0,12.0,0.750,0.0,1.0,0.000,3.0,4.0,0.75,1.0,2.0,3.0,4.0,0.0,0.0,2.0,2.0,21.0,-10.0
7,40.250000,2.0,13.0,0.154,1.0,7.0,0.143,5.0,5.0,1.00,0.0,3.0,3.0,4.0,1.0,2.0,3.0,4.0,10.0,8.0
8,0.000000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [427]:
from torch.nn import LSTM

# forecasting performance of team A
torch.manual_seed(4188)
a_rnn = LSTM(input_size=num_quantities, hidden_size=num_quantities).to(device)
a_h0 = Tr((1,num_players,num_quantities))
a_c0 = Tr((1,num_players,num_quantities))
output, (a_hn, a_cn) = a_rnn(box_scores, (a_h0, a_c0))